In [107]:
import pandas as pd
import numpy as np
import os
import re

In [108]:
df_nba_personal = pd.read_csv('dataset/raw/nba_personal.csv')
df_nba_stats = pd.read_csv('dataset/raw/nba_stats.csv')
df_nba_experience = pd.read_json('dataset/raw/nba_experience.json')
df_nba_season = pd.read_excel('dataset/raw/nba_season.xlsx')

In [109]:
df_nba_personal.head()

,Unnamed: 0,playerId,playerSlug,college,draftinfo,htwt,name,birthdate,birthplace,year
0,0,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)","nan, nan",Allen Iverson,6/7/1975,"Hampton, VA",1999
1,1,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)","nan, nan",Allen Iverson,6/7/1975,"Hampton, VA",2000
2,2,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)","nan, nan",Allen Iverson,6/7/1975,"Hampton, VA",2001
3,3,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)","nan, nan",Allen Iverson,6/7/1975,"Hampton, VA",2002
4,4,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)","nan, nan",Allen Iverson,6/7/1975,"Hampton, VA",2003


In [110]:
df_nba_personal['height'] = df_nba_personal['htwt'].apply(
    lambda x: x.split(',')[1].strip() if pd.notnull(x) else x)

df_nba_personal['weight'] = df_nba_personal['htwt'].apply(
    lambda x: x.split(',')[0].strip() if pd.notnull(x) else x)

df_nba_personal = df_nba_personal.drop(
    ['htwt', df_nba_personal.columns[0]], axis=1)

In [111]:
df_nba_personal.head()

,playerId,playerSlug,college,draftinfo,name,birthdate,birthplace,year,height,weight
0,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,"Hampton, VA",1999,nan,nan
1,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,"Hampton, VA",2000,nan,nan
2,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,"Hampton, VA",2001,nan,nan
3,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,"Hampton, VA",2002,nan,nan
4,366,allen-iverson,Georgetown,"1996: Rd 1, Pk 1 (PHI)",Allen Iverson,6/7/1975,"Hampton, VA",2003,nan,nan


In [112]:
df_nba_stats.head()

,playerId,gamesPlayed,avgMinutes,avgFouls,flagrantFouls,technicalFouls,ejections,doubleDouble,tripleDouble,minutes,...,freeThrowsMade,freeThrowsAttempted,assists,turnovers,avgSteals,avgBlocks,steals,blocks,totalPoints,year
0,366,48.0,41.458332,2.0416667,0.0,5.0,0.0,4.0,0.0,1990.0,...,356.0,474.0,223.0,167.0,2.291667,0.145833,110.0,7.0,994.999968,1999
1,366,70.0,40.757140,2.3142858,0.0,12.0,0.0,1.0,0.0,2853.0,...,442.0,620.0,328.0,230.0,2.057143,0.071429,144.0,5.0,99999.000000,2000
2,366,71.0,NaN,2.0704226,0.0,18.0,0.0,4.0,0.0,2979.0,...,585.0,719.0,325.0,237.0,2.507042,0.281690,178.0,20.0,NaN,2001
3,366,60.0,NaN,dois,0.0,12.0,1.0,4.0,1.0,2622.0,...,475.0,585.0,331.0,237.0,2.800000,0.216667,168.0,13.0,NaN,2002
4,366,82.0,NaN,1.8170732,zero,8.0,0.0,6.0,0.0,3485.0,...,570.0,736.0,454.0,286.0,2.743902,0.158537,225.0,13.0,NaN,2003


In [113]:
for idx,avgMinutes in enumerate(df_nba_stats["avgMinutes"]):
    if(type(avgMinutes) is str):
        df_nba_stats.loc[idx, 'avgMinutes'] = avgMinutes.replace(',', '.')
        df_nba_stats.loc[idx, 'avgMinutes'] = avgMinutes.replace(' ', '.')

        
for idx, avgFouls in enumerate(df_nba_stats["avgFouls"]):
    if (type(avgFouls) is str):
        df_nba_stats.loc[idx, 'avgFouls'] = avgFouls.replace(',','.')
        if avgFouls == 'dois':
            df_nba_stats.loc[idx, 'avgFouls'] = '2'
            
for idx, flagrantFouls in enumerate(df_nba_stats["flagrantFouls"]):
    if (type(flagrantFouls) is str):
        df_nba_stats.loc[idx, 'flagrantFouls'] = flagrantFouls.replace(
            ',', '.')
        if flagrantFouls == 'zero':
            df_nba_stats.loc[idx, 'flagrantFouls'] = '0'

for idx, playerId in enumerate(df_nba_stats["playerId"]):
    if (type(playerId) is str):
        if playerId == 'threehundred66':
            df_nba_stats.loc[idx, 'playerId'] = '366'
        
        
df_nba_stats["avgMinutes"] = df_nba_stats["avgMinutes"].astype(float)
df_nba_stats["avgFouls"] = df_nba_stats["avgFouls"].astype(float)
df_nba_stats["flagrantFouls"] = df_nba_stats["flagrantFouls"].astype(float)
df_nba_stats["playerId"] = df_nba_stats["playerId"].astype(int)

In [114]:
df_nba_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164 entries, 0 to 8163
Data columns (total 40 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   playerId                          8164 non-null   int32  
 1   gamesPlayed                       8163 non-null   float64
 2   avgMinutes                        8161 non-null   float64
 3   avgFouls                          8164 non-null   float64
 4   flagrantFouls                     8164 non-null   float64
 5   technicalFouls                    8164 non-null   float64
 6   ejections                         8164 non-null   float64
 7   doubleDouble                      8164 non-null   float64
 8   tripleDouble                      8164 non-null   float64
 9   minutes                           8164 non-null   float64
 10  rebounds                          8164 non-null   float64
 11  fouls                             8164 non-null   float64
 12  avgReb

In [115]:
df_nba_experience.head()

,playerId,salary,rankingSalary,experience,year
0,366,"$9,000,000",15,None,1999
1,366,"$10,125,000",27,None,2000
2,366,"$11,250,000",17,None,2001
3,366,"$12,375,000",14,None,2002
4,366,"$13,500,000",11,None,2003


In [116]:
for idx, salary in enumerate(df_nba_experience["salary"]):
    if (type(salary) is str):
        value = salary.replace('$','').replace(',', '').strip()
        df_nba_experience.loc[idx, 'salary'] = value
        
df_nba_experience["salary"] = df_nba_experience["salary"].astype(float)

In [117]:
df_nba_experience.head()

,playerId,salary,rankingSalary,experience,year
0,366,9000000.0,15,None,1999
1,366,10125000.0,27,None,2000
2,366,11250000.0,17,None,2001
3,366,12375000.0,14,None,2002
4,366,13500000.0,11,None,2003


In [118]:
df_nba_season.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8164 entries, 0 to 8163
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   playerId    8164 non-null   object
 1   positionId  8158 non-null   object
 2   position    8164 non-null   object
 3   teamId      8164 non-null   object
 4   team        2304 non-null   object
 5   status      8164 non-null   object
 6   year        8164 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 446.6+ KB


In [119]:
df_nba_season.head()

,playerId,positionId,position,teamId,team,status,year
0,366,SG,Shooting Guard,PHI,NaN,inactive,1999
1,366,SG,Shooting Guard,PHI,NaN,inactive,2000
2,366,SG,Shooting Guard,PHI,NaN,inactive,2001
3,366,SG,Shooting Guard,PHI,NaN,inactive,2002
4,366,SG,Shooting Guard,PHI,NaN,inactive,2003


In [120]:
os.mkdir('./processed')
df_nba_experience.to_csv("./processed/nba_experience.csv", index=False)
df_nba_personal.to_csv("./processed/nba_personal.csv", index=False)
df_nba_season.to_csv("./processed/nba_season.csv", index=False)
df_nba_stats.to_csv("./processed/nba_stats.csv", index=False)